In [38]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os

In [39]:
load_dotenv()
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

In [40]:
import minsearch
import json
import textwrap

# wrapped_text = textwrap.fill(documents[0]["text"], width=80)

In [41]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [42]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [43]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [44]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course'], 
)

import textwrap
wrapped_texct = textwrap.fill(documents[0]['text'], width=80)
print(wrapped_texct)

In [46]:
index.fit(documents)

In [56]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5,
    )

    return results


In [57]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """
    context = ""

    for doc in search_results:
        context = (
            context
            + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        )

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [58]:
def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
    )

    response.choices[0].message.content
    return response.choices[0].message.content

In [59]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    response = llm(prompt)
    print(response)
    return response

query = "how do I run kafka?"

To run Kafka, you can follow the instructions based on your specific use case. Here are the steps for different scenarios:

1. **Java Kafka (Producer/Consumer/KStreams):**
   - Navigate to your project directory.
   - Run the following command in the terminal:
     ```bash
     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
     ```

2. **Python Kafka (when module “kafka” is not found):**
   - Create a virtual environment and install the required packages using the following commands:
     ```bash
     python -m venv env
     source env/bin/activate  # On MacOS/Linux
     # On Windows use: env\Scripts\activate
     pip install -r ../requirements.txt
     ```
   - To activate the virtual environment (each time before running your Python files):
     ```bash
     source env/bin/activate  # On MacOS/Linux
     # On Windows use: env\Scripts\activate
     ```
   - To deactivate it:
     ```bash
     deactivate
     ```

3. **Fixing "Permissio